In [19]:
# !pip install llama-index
# !pip install llama-index-embeddings-llamafile
# !pip install llama-index-llms-llamafil
# !pip install llama-index-readers-web


In [6]:
# chmod +x
# ./TinyLlama-1.1B-Chat-v1.0.F16.llamafile --server --nobrowser --embedding --port 8080
# ./Phi-3-mini-4k-instruct.F16.llamafile --server --nobrowser --embedding --port 8080

In [20]:
# Configure LlamaIndex
from llama_index.core import Settings
from llama_index.embeddings.llamafile import LlamafileEmbedding
from llama_index.llms.llamafile import Llamafile
from llama_index.core.node_parser import SentenceSplitter

Settings.embed_model = LlamafileEmbedding(base_url="http://localhost:8080")

Settings.llm = Llamafile(
	base_url="http://localhost:8080",
	temperature=0,
	seed=0
)

# Also set up a sentence splitter to ensure texts are broken into semantically-meaningful chunks (sentences) that don't take up the model's entire
# context window (2048 tokens). Since these chunks will be added to LLM prompts as part of the RAG process, we want to leave plenty of space for both
# the system prompt and the user's actual question.
Settings.transformations = [
	SentenceSplitter(
    	chunk_size=256,
    	chunk_overlap=5
	)
]

In [15]:
# Load local data
from llama_index.core import SimpleDirectoryReader
local_doc_reader = SimpleDirectoryReader(input_dir='./data')
docs = local_doc_reader.load_data(show_progress=True)

# We'll load some Wikipedia pages as well
from llama_index.readers.web import SimpleWebPageReader
urls = [
	'https://en.wikipedia.org/wiki/Push-up',
]
web_reader = SimpleWebPageReader(html_to_text=True)
docs.extend(web_reader.load_data(urls))

# Build the index
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(
	docs,
	show_progress=True,
)



Loading files: 100%|██████████| 2/2 [00:00<00:00, 141.22file/s]
/opt/anaconda3/envs/llamaindex/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 706/706 [06:31<00:00,  1.81it/s]


In [21]:
# Save the index
index.storage_context.persist(persist_dir="./storage")

In [22]:
query_engine = index.as_query_engine()


In [23]:
question = "what are some of the variations of push-ups?"

In [24]:
print(query_engine.query(question))

ReadTimeout: timed out

### CalorieNinjas API

In [5]:
import requests
import os
from dotenv import load_dotenv


In [6]:
load_dotenv()
X_API_KEY = os.getenv('CALORIE_API_KEY')

In [7]:
api_url = 'https://api.calorieninjas.com/v1/nutrition?query='
query = '3lb carrots and a chicken sandwich'

In [12]:
def get_nutrition(query):
    response = requests.get(api_url + query, headers={'X-Api-Key': X_API_KEY})
    if response.status_code == requests.codes.ok:
        return response.json()
    else:
        return f"Error: {response.status_code} {response.text}"

In [13]:
print(get_nutrition("3lb carrots and a chicken sandwich"))

{'items': [{'name': 'carrots', 'calories': 477.8, 'serving_size_g': 1360.7759999999998, 'fat_total_g': 2.3, 'fat_saturated_g': 0.3, 'protein_g': 10.3, 'sodium_mg': 775, 'potassium_mg': 410, 'cholesterol_mg': 0, 'carbohydrates_total_g': 110.2, 'fiber_g': 40.7, 'sugar_g': 46.7}, {'name': 'chicken sandwich', 'calories': 243.9, 'serving_size_g': 100.0, 'fat_total_g': 11.2, 'fat_saturated_g': 2.1, 'protein_g': 16.2, 'sodium_mg': 767, 'potassium_mg': 184, 'cholesterol_mg': 35, 'carbohydrates_total_g': 20.9, 'fiber_g': 1.4, 'sugar_g': 3.6}]}


{
  "items": [
    {
      "name": "carrots",
      "calories": 477.8,
      "serving_size_g": 1360.776,
      "fat_total_g": 2.3,
      "fat_saturated_g": 0.3,
      "protein_g": 10.3,
      "sodium_mg": 775,
      "potassium_mg": 410,
      "cholesterol_mg": 0,
      "carbohydrates_total_g": 110.2,
      "fiber_g": 40.7,
      "sugar_g": 46.7
    },
    {
      "name": "chicken sandwich",
      "calories": 243.9,
      "serving_size_g": 100.0,
      "fat_total_g": 11.2,
      "fat_saturated_g": 2.1,
      "protein_g": 16.2,
      "sodium_mg": 767,
      "potassium_mg": 184,
      "cholesterol_mg": 35,
      "carbohydrates_total_g": 20.9,
      "fiber_g": 1.4,
      "sugar_g": 3.6
    }
  ]
}
